# Merging Datasets
Sometimes we don't have the luxury of a well prepared Kaggle dataset. Sometimes we may have to work with a database dump, with separate files describing separate tables of database. And we have to stitch them together ourselves. So that's what we're doing today!\

Here's a very simple dataset to practice on : http://tiny.cc/studcsv \
Let's find all students that have failed three or more courses and send their parents an email from their teachers.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
students = pd.read_csv(r'students.csv')
teachers = pd.read_csv(r'teachers.csv')
grades1 = pd.read_csv(r'grades1.csv')
grades2 = pd.read_csv(r'grades2.csv')
contacts = pd.read_csv(r'contact.csv')

In [7]:
students.head(2)

,id,firstname,lastname
0,34292,Joshua,Davis
1,34293,Karen,Flores


In [9]:
teachers.head(2)

,course,teacher
0,PHYS101,Wayne Walker
1,CHEM101,Albert Taylor


In [10]:
grades1.head(2)

,student_id,course,grade
0,34292,ENGL101,A
1,34293,ENGL101,A


In [11]:
grades2.head(2)

,student_id,course,grade
0,34302,DRAM101,F
1,34303,DRAM101,D


In [12]:
contacts.head(2)

,student_id,parent_contact,phone
0,34292,Samuel Davis,(356) 849-0352
1,34293,Laura Flores,(477) 325-7117


## Concat and append

In [13]:
grades1['Semester'] = 1
grades2['Semester'] = 2

grades = pd.concat([grades1, grades2])
grades

,student_id,course,grade,Semester
0,34292,ENGL101,A,1
1,34293,ENGL101,A,1
2,34294,ENGL101,C,1
3,34295,ENGL101,C,1
4,34296,ENGL101,C,1
...,...,...,...,...
60,34312,DESN101,E,2
61,34313,DESN101,D,2
62,34314,DESN101,A,2
63,34315,DESN101,A,2


In [14]:
grades = grades.reset_index(drop=True)
grades

,student_id,course,grade,Semester
0,34292,ENGL101,A,1
1,34293,ENGL101,A,1
2,34294,ENGL101,C,1
3,34295,ENGL101,C,1
4,34296,ENGL101,C,1
...,...,...,...,...
185,34312,DESN101,E,2
186,34313,DESN101,D,2
187,34314,DESN101,A,2
188,34315,DESN101,A,2


That's better. Not that it would have caused us any issues in this example. Now let's actually merge these dataframes together. Let's put the student information in, merging on the student id.

## Merging horizontally

In [15]:
student_grades = pd.merge(students, grades, left_on='id', right_on='student_id')
student_grades.head()

,id,firstname,lastname,student_id,course,grade,Semester
0,34292,Joshua,Davis,34292,ENGL101,A,1
1,34292,Joshua,Davis,34292,MATH101,A,1
2,34292,Joshua,Davis,34292,PHYS101,D,1
3,34292,Joshua,Davis,34292,MATH102,F,1
4,34292,Joshua,Davis,34292,COMP101,A,1


In [17]:
student_grades

,id,firstname,lastname,student_id,course,grade,Semester
0,34292,Joshua,Davis,34292,ENGL101,A,1
1,34292,Joshua,Davis,34292,MATH101,A,1
2,34292,Joshua,Davis,34292,PHYS101,D,1
3,34292,Joshua,Davis,34292,MATH102,F,1
4,34292,Joshua,Davis,34292,COMP101,A,1
...,...,...,...,...,...,...,...
185,34316,Stephen,Martinez,34316,ARTT101,A,1
186,34316,Stephen,Martinez,34316,DESN101,A,1
187,34316,Stephen,Martinez,34316,DRAM101,F,2
188,34316,Stephen,Martinez,34316,ARTT101,A,2


In [18]:
print(students.shape, grades.shape, student_grades.shape)

(25, 3) (190, 4) (190, 7)


In [23]:
students2 = students.rename({'id': 'student_id'}, axis=1)
students_full = students2.merge(contacts, on='student_id')
students_full.head()

,student_id,firstname,lastname,parent_contact,phone
0,34292,Joshua,Davis,Samuel Davis,(356) 849-0352
1,34293,Karen,Flores,Laura Flores,(477) 325-7117
2,34294,Julia,Walker,Eric Walker,(871) 639-0797
3,34295,Cynthia,Robinson,Chris Robinson,(574) 683-2107
4,34296,Bonnie,Hall,Dorothy Hall,(384) 293-2113


In [24]:
student_grades = pd.merge(students2, grades, on='student_id')
df = pd.merge(student_grades, contacts, on='student_id')
df.head()

,student_id,firstname,lastname,course,grade,Semester,parent_contact,phone
0,34292,Joshua,Davis,ENGL101,A,1,Samuel Davis,(356) 849-0352
1,34292,Joshua,Davis,MATH101,A,1,Samuel Davis,(356) 849-0352
2,34292,Joshua,Davis,PHYS101,D,1,Samuel Davis,(356) 849-0352
3,34292,Joshua,Davis,MATH102,F,1,Samuel Davis,(356) 849-0352
4,34292,Joshua,Davis,COMP101,A,1,Samuel Davis,(356) 849-0352


In [26]:
df = df.merge(teachers, on='course')
df.head()

,student_id,firstname,lastname,course,grade,Semester,parent_contact,phone,teacher
0,34292,Joshua,Davis,ENGL101,A,1,Samuel Davis,(356) 849-0352,Ernest Green
1,34292,Joshua,Davis,MATH101,A,1,Samuel Davis,(356) 849-0352,Judy Garcia
2,34292,Joshua,Davis,PHYS101,D,1,Samuel Davis,(356) 849-0352,Wayne Walker
3,34292,Joshua,Davis,MATH102,F,1,Samuel Davis,(356) 849-0352,Samuel Edwards
4,34292,Joshua,Davis,COMP101,A,1,Samuel Davis,(356) 849-0352,Jane Simmons


How many classes that student failed

In [35]:
df2 = df.loc[df.grade == 'F', 'student_id']
method1 = df2.value_counts().rename_axis('student_id').reset_index(name='counts')
method1

,student_id,counts
0,34300,4
1,34302,3
2,34316,2
3,34292,2
4,34294,2
5,34299,2
6,34301,2
7,34303,2
8,34309,2
9,34304,2


In [41]:
method2 = df.loc[df.grade == 'F', ['student_id', 'grade']].groupby(by='student_id').count().reset_index()
method2.columns = ['student_id','count']
method2

,student_id,count
0,34292,2
1,34293,1
2,34294,2
3,34297,1
4,34298,1
5,34299,2
6,34300,4
7,34301,2
8,34302,3
9,34303,2


In [42]:
sids = method1[method1.counts >= 3].student_id
sids.head()

0    34300
1    34302
Name: student_id, dtype: int64

In [45]:
final = students_full[students_full.student_id.isin(sids)]
final

,student_id,firstname,lastname,parent_contact,phone
8,34300,Mary,Mitchell,Edward Mitchell,(494) 471-1604
10,34302,Dorothy,Green,Russell Green,(749) 344-5744


## Join
What about pd.join? It's a specific version of merge that works solely more on index

In [47]:
display(students.head(), contacts.head())

,id,firstname,lastname
0,34292,Joshua,Davis
1,34293,Karen,Flores
2,34294,Julia,Walker
3,34295,Cynthia,Robinson
4,34296,Bonnie,Hall


,student_id,parent_contact,phone
0,34292,Samuel Davis,(356) 849-0352
1,34293,Laura Flores,(477) 325-7117
2,34294,Eric Walker,(871) 639-0797
3,34295,Chris Robinson,(574) 683-2107
4,34296,Dorothy Hall,(384) 293-2113


In [48]:
students3 = students.set_index('id')
contacts3 = contacts.set_index('student_id')
students3.join(contacts3)

,firstname,lastname,parent_contact,phone
id,,,,
34292,Joshua,Davis,Samuel Davis,(356) 849-0352
34293,Karen,Flores,Laura Flores,(477) 325-7117
34294,Julia,Walker,Eric Walker,(871) 639-0797
34295,Cynthia,Robinson,Chris Robinson,(574) 683-2107
34296,Bonnie,Hall,Dorothy Hall,(384) 293-2113
34297,Gary Lee,Lee,Mary Lee,(914) 452-6366
34298,Ruby,Thompson,Jose Thompson,(253) 419-7702
34299,Heather,Miller,Annie Miller,(864) 583-9615
34300,Mary,Mitchell,Edward Mitchell,(494) 471-1604


## Recap
- concat
- append
- merge(inner, left, right, outer)
- join